# Results inspection for EV dispatch
Inspect and visualize patterns from EV dispatch optimization

## Package imports

In [ ]:
import pandas as pd
from pommesevaluation.electric_vehicles_dispatch import draw_weekly_plot

## Notebook and workflow settings

In [ ]:
# Simulation settings
time_frame_in_years = 1
freq = "1H"
multiplier = {
    "1H": 1,
    "4H": 4,
    "8H": 8,
}
fuel_cost_pathway = "NZE"
fuel_price_shock = "high"
emissions_cost_pathway = "long-term"
impose_investment_maxima = True
dr_scenario = "50"

# Paths to data needed to be converted
path_model_inputs = "./model_inputs/pommesinvest/"
path_model_results = "./model_results/pommesinvest/"

# Map corresponding flex options scenario (values) to dr scenarios (keys)
dr_scenarios = {
    "none": "50",
    "5": "5", 
    "50": "50",
    "95": "95"
}

if impose_investment_maxima:
    annual_investment_limits = ""
else:
    annual_investment_limits = "_no_annual_limit_import_costs"
file_names_dispatch_results = {
    scen: (
        f"investment_LP_start-2020-01-01_{time_frame_in_years}"
        f"-years_simple_freq_{freq}_with_dr_{scen}_"
        f"fuel_price-{fuel_cost_pathway}_{fuel_price_shock}_"
        f"co2_price-{emissions_cost_pathway}{annual_investment_limits}_production.csv"
    ) for scen in dr_scenarios
}
file_names_dispatch_results["none"] = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{freq}_no_dr_50_"
    f"fuel_price-{fuel_cost_pathway}_{fuel_price_shock}_"
    f"co2_price-{emissions_cost_pathway}{annual_investment_limits}_production.csv"
)
dispatch_multi_index = False

file_name_electric_vehicles = f"components_electric_vehicles_{dr_scenario}.csv"
file_name_electric_vehicles_ts = f"electric_vehicles_ts_{dr_scenario}.csv"

## Read in and filter electric vehicles dispatch results

In [ ]:
if dispatch_multi_index:
    header = [0, 1]
else:
    header = 0

dispatch_results = pd.read_csv(
    f"{path_model_results}{file_names_dispatch_results[dr_scenario]}", index_col=0, header=header
)
ev_dispatch = dispatch_results[[
    col for col in dispatch_results.columns if "ev_" in col or "_ev" in col
]]

In [ ]:
ev_dispatch["cc_net_flow"] = ev_dispatch[
    "(('DE_bus_el', 'transformer_ev_cc_bidirectional_consumption'), 'flow')"
] + ev_dispatch[
    "(('DE_bus_el', 'transformer_ev_cc_unidirectional'), 'flow')"
] - ev_dispatch[
    "(('transformer_ev_cc_bidirectional_generation', 'DE_bus_el'), 'flow')"
]
ev_dispatch["net_storage"] = ev_dispatch["(('DE_bus_ev', 'storage_ev'), 'flow')"] - ev_dispatch[
    "(('storage_ev', 'DE_bus_ev'), 'flow')"
]
# Extract ev storage level pattern
ev_dispatch["storage_cumsum"] = ev_dispatch["net_storage"].cumsum()
ev_storage_initial = -ev_dispatch["storage_cumsum"].min()
ev_dispatch["storage_cumsum"] += ev_storage_initial

ev_capacity = ev_dispatch["storage_cumsum"].max() - ev_dispatch["storage_cumsum"].min()
print(ev_capacity)

In [ ]:
ev_dispatch.columns

In [ ]:
draw_weekly_plot(
    ev_dispatch[[
        "cc_net_flow", "net_storage", 
        "(('DE_bus_ev', 'sink_ev_driving'), 'flow')", 
        "(('DE_bus_el', 'transformer_ev_uc'), 'flow')",
        "(('DE_bus_el', 'transformer_ev_cc_unidirectional'), 'flow')",
    ]], 2020
)

## Read in parameterization

In [ ]:
electric_vehicles = pd.read_csv(
    f"{path_model_inputs}{file_name_electric_vehicles}", index_col=0
)
electric_vehicles_ts = pd.read_csv(
    f"{path_model_inputs}{file_name_electric_vehicles_ts}", index_col=0
)

In [ ]:
electric_vehicles_ts.loc[electric_vehicles_ts.index.str[:4] == "2020"].min()

In [ ]:
# Calculate storage capacity from lower and upper bounds (max - min)
(electric_vehicles.at["storage_ev", "nominal_value"] * (
    electric_vehicles_ts.loc[electric_vehicles_ts.index.str[:4] == "2020"].max().loc["soc_upper"]
) - electric_vehicles.at["storage_ev", "nominal_value"] * (
    electric_vehicles_ts.loc[electric_vehicles_ts.index.str[:4] == "2020"].min().loc["soc_lower"]
))